# AIOE 계산 구현

이 노트북에서는 전처리된 데이터를 사용하여 각 직업의 **AI Occupational Exposure (AIOE)** 점수를 계산합니다.

## AIOE란?

AIOE는 특정 직업이 AI 기술(특히 Language Modeling)에 얼마나 노출되어 있는지를 측정하는 지표입니다.
- **높은 AIOE**: AI가 해당 직업의 많은 능력을 대체하거나 보완할 수 있음
- **낮은 AIOE**: AI가 해당 직업의 능력을 대체하기 어려움

In [567]:
import pandas as pd
import numpy as np
from scipy import stats

In [568]:
# 데이터 경로 설정
PROCESSED_DIR = '../datas/processed'

## 1. 전처리된 데이터 불러오기

### 이전 단계에서 우리가 만든 데이터

01번 노트북에서 우리는 2개의 데이터를 합쳐서 하나의 파일을 만들었습니다:
- **O*NET Abilities**: 894개 직업 × 52개 능력의 Importance, Level
- **MTurk 매핑**: 52개 능력의 Beta 값 (AI 영향도)

**최종 데이터 구조:**
- **46,488개 행** (894 직업 × 52 능력)
- **6개 컬럼**: SOC_Code, Title, Ability, Importance, Level, Beta

**이제 할 일:**
이 데이터를 사용해서 각 직업의 **AIOE 점수를 계산**합니다.
- 입력: 46,488개 행 (직업-능력 조합)
- 출력: 894개 행 (직업별 AIOE 점수)

In [569]:
# 전처리된 데이터 로드
df = pd.read_csv(f'{PROCESSED_DIR}/preprocessed_data.csv')

print(f"✅ 데이터 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}컬럼, {df['soc_code'].nunique()}개 직업")

✅ 데이터 로드 완료: 46,488행 × 6컬럼, 894개 직업


## 2. AIOE 계산 아이디어

### 우리가 해결하려는 문제

**질문:** "소프트웨어 개발자가 AI에 얼마나 노출되어 있을까?"

**가지고 있는 정보:**
1. 소프트웨어 개발자에게 필요한 52개 능력 (Importance, Level)
2. AI가 각 능력을 얼마나 잘하는지 (Beta)

### 계산 아이디어

**핵심 질문:**
> "전체 능력 중에서 AI가 도와줄 수 있는 능력이 차지하는 비중은?"

**실제 Software Developers 데이터 5개로 살펴봅시다:**

| 능력 (한국어) | Importance | Level | Scalar<br/>(= I × L) | Beta | 가중 영향도<br/>(= Scalar × Beta) |
|--------------|-----------|-------|---------|------|------------------|
| 듣고 이해하기<br/>(Oral Comprehension) | 3.75 | 4.12 | **15.45** | 0.906 | 13.998 |
| 글 읽고 이해하기<br/>(Written Comprehension) | 3.75 | 4.38 | **16.425** | 0.827 | 13.584 |
| 말로 표현하기<br/>(Oral Expression) | 3.75 | 4.12 | **15.45** | 0.906 | 13.998 |
| 글로 표현하기<br/>(Written Expression) | 3.50 | 4.00 | **14.00** | 0.848 | 11.872 |
| 아이디어 떠올리기<br/>(Fluency of Ideas) | 3.25 | 3.62 | **11.765** | 0.717 | 8.436 |
| **5개 능력 합계** | - | - | **73.09** | - | **61.888** |

**컬럼 의미:**
1. **Importance**: 이 능력이 직업에서 얼마나 중요한가 (1~5 척도)
2. **Level**: 이 능력이 얼마나 높은 수준을 요구하는가 (0~7 척도)
3. **Scalar** (I × L): 능력의 가중치 (중요도 × 수준)
4. **Beta**: AI가 이 능력을 얼마나 잘하는가 (0~1, MTurk 데이터)
5. **가중 영향도** (Scalar × Beta): AI가 이 능력에서 도와줄 수 있는 정도

**직접 계산해볼까요?**
- 듣고 이해하기 Scalar = 3.75 × 4.12 = **15.45** 
- 가중 영향도 = 15.45 × 0.906 = **13.998** 

**5개 능력만으로 AIOE를 계산하면:**
- AIOE = 61.888 ÷ 73.09 = **0.847 (84.7%)**
- 이 5개 능력만 보면 AI가 84.7% 도와줄 수 있다는 의미

**그런데 문제가 있습니다!**
- Importance는 1~5 척도, Level은 0~7 척도
- 서로 다른 척도를 곱하면 공정하지 않습니다
- **해결책**: 다음 단계에서 0~1로 정규화합니다

**실제로는 52개 능력 전체를 사용:**
- 위 예시는 이해를 돕기 위한 5개만 선택
- 실제 계산에서는 52개 능력 모두 사용
- 하지만 계산 방식은 동일합니다!

### 계산 순서 (5단계)

우리는 이제 **단계별로** 계산해 나갈 것입니다:

1. **정규화**: 서로 다른 척도를 0-1로 통일
2. **Scalar 계산**: 정규화된 값으로 능력 가중치 계산
3. **가중 영향도 계산**: 각 능력에서 AI의 기여도
4. **직업별 집계**: 52개 능력을 모두 합산
5. **표준화**: 다른 직업과 비교 가능하도록 변환

**지금은 복잡해 보이지만, 한 단계씩 같이 해볼게요!**

---

## 3. Step 1: Importance와 Level을 0-1 범위로 변환하기

### 왜 변환이 필요한가요?

**현재 문제:**
- Importance: 1~5 척도
- Level: 0~7 척도
→ **서로 다른 척도를 곱하면 공정하지 않음!**

**해결책: 정규화 (Normalization)**
- Importance: 원래 값 ÷ 5  (예: 3.5 → 0.70)
- Level: 원래 값 ÷ 7  (예: 3.88 → 0.55)

**결과:**
- 모든 값이 0~1 사이로 변환
- 공정한 비교 가능
- 0 = 최소, 1 = 최대

In [ ]:
# TODO: Importance와 Level을 0-1 범위로 정규화하세요
#
# 힌트 1: Importance는 1-5 척도이므로 5로 나누기
# 힌트 2: Level은 0-7 척도이므로 7로 나누기
# 힌트 3: 새 컬럼명은 scale_importance, scale_level
#
# 새 컬럼명: scale_importance, scale_level

# 여기에 코드를 작성하세요



In [571]:
# 변환 결과 확인
df[df['title'] == 'Software Developers'][
    ['ability', 'importance', 'scale_importance', 'level', 'scale_level']
].head()

,ability,importance,scale_importance,level,scale_level
5824,Oral Comprehension,3.75,0.75,4.12,0.588571
5825,Written Comprehension,3.75,0.75,4.38,0.625714
5826,Oral Expression,3.75,0.75,4.12,0.588571
5827,Written Expression,3.50,0.70,4.00,0.571429
5828,Fluency of Ideas,3.25,0.65,3.62,0.517143


In [572]:
df[df['title'] == 'Software Developers'][
    ['scale_importance', 'scale_level']
].describe()

,scale_importance,scale_level
count,52.000000,52.000000
mean,0.424500,0.252198
std,0.206691,0.223643
min,0.200000,0.000000
25%,0.200000,0.000000
50%,0.363000,0.205714
75%,0.624000,0.468929
max,0.750000,0.625714


## 4. Step 2: Scalar 계산하기 (정규화 후)

### 이제 정규화된 값으로 Scalar를 계산합니다

**Scalar = scale_level × scale_importance**

**정규화 전 vs 후 비교:**

| 능력 | 정규화 전<br/>I × L | 정규화 후<br/>I × L | 
|------|-----------------|-----------------|
| Written Comprehension | 3.75 × 4.38 = **16.425** | 0.75 × 0.626 = **0.469** |
| Fluency of Ideas | 3.25 × 3.62 = **11.765** | 0.65 × 0.517 = **0.338** |

**결론:**
- **Scalar가 클수록** = 그 직업의 핵심 능력
- **Scalar가 작을수록** = 덜 중요한 능력

In [ ]:
# TODO: Scalar를 계산하세요 (능력 가중치)
#
# 힌트: Scalar = scale_level × scale_importance
#
# 새 컬럼명: scalar

# 여기에 코드를 작성하세요



In [574]:
# Scalar 상위 능력 확인
df[df['title'] == 'Software Developers'][
    ['ability', 'scale_importance', 'scale_level', 'scalar']
].nlargest(10, 'scalar')

,ability,scale_importance,scale_level,scalar
5825,Written Comprehension,0.750,0.625714,0.469286
5824,Oral Comprehension,0.750,0.588571,0.441429
5826,Oral Expression,0.750,0.588571,0.441429
5831,Deductive Reasoning,0.750,0.571429,0.428571
5830,Problem Sensitivity,0.750,0.554286,0.415714
5864,Near Vision,0.750,0.535714,0.401786
5833,Information Ordering,0.724,0.554286,0.401303
5827,Written Expression,0.700,0.571429,0.400000
5832,Inductive Reasoning,0.700,0.554286,0.388000
5834,Category Flexibility,0.700,0.535714,0.375000


## 5. Step 3: AI 가중 영향도 계산하기

### 가중 영향도란?

**가중 영향도 = Scalar × Beta**

**의미:**
- Scalar: 이 능력이 직업에서 얼마나 중요한가? (능력의 가중치)
- Beta: 이 능력을 AI가 얼마나 잘하는가? (AI 영향도)
- **가중 영향도**: AI가 **실제로** 이 직업에 미치는 영향

**핵심 아이디어:**
```
능력이 중요해도 AI가 못하면 → 영향 없음
AI가 잘해도 능력이 안 중요하면 → 영향 없음
둘 다 높아야 → 큰 영향!
```

**구체적인 상황:**

| 능력 | Scalar | Beta | 가중 영향도 | 해석 |
|------|----------|---------|-----------|------|
| Written Comprehension | 0.60 | 0.83 | **0.50** | 🔥 AI가 큰 영향 |
| Manual Dexterity | 0.08 | 0.20 | **0.02** | ✅ AI 영향 없음 |

In [ ]:
# TODO: 가중 영향도를 계산하세요
#
# 힌트: 가중 영향도 = scalar × beta
#
# 새 컬럼명: wtd_element_impact

# 여기에 코드를 작성하세요



In [576]:
# AI 영향도가 높은 능력 TOP 10
df[df['title'] == 'Software Developers'][
    ['ability', 'scalar', 'beta', 'wtd_element_impact']
].nlargest(10, 'wtd_element_impact')

,ability,scalar,beta,wtd_element_impact
5824,Oral Comprehension,0.441429,0.905759,0.399828
5826,Oral Expression,0.441429,0.905759,0.399828
5825,Written Comprehension,0.469286,0.827225,0.388205
5831,Deductive Reasoning,0.428571,0.801047,0.343306
5833,Information Ordering,0.401303,0.848168,0.340372
5827,Written Expression,0.400000,0.848168,0.339267
5832,Inductive Reasoning,0.388000,0.785340,0.304712
5874,Speech Recognition,0.313857,0.952880,0.299068
5834,Category Flexibility,0.375000,0.759162,0.284686
5875,Speech Clarity,0.278571,0.926702,0.258153


---

## 6. Step 4: 직업별 집계 및 AIOE 기본 점수 계산

### 이제 모든 조각을 합칠 시간입니다!

**지금까지 계산한 것:**
- ✅ Scalar (각 능력의 가중치)
- ✅ 가중 영향도 (AI가 각 능력에서 도와줄 수 있는 비중)

**이제 할 일:**
- 52개 능력을 모두 합산해서 직업별 AIOE 계산

### AIOE 공식

$$
AIOE_j = \frac{\sum_{a=1}^{52} (L_{ja} \times I_{ja} \times \beta_a)}{\sum_{a=1}^{52} (L_{ja} \times I_{ja})}
$$

**공식 분해:**
- $\sum_{a=1}^{52} (L_{ja} \times I_{ja} \times \beta_a)$ = **분자**: 52개 능력의 가중 영향도 합 (AI가 도와줄 수 있는 총 비중)
- $\sum_{a=1}^{52} (L_{ja} \times I_{ja})$ = **분모**: 52개 능력의 Scalar 합 (전체 능력 가중치 = 100%)
- **결과**: AI가 도와줄 수 있는 비중 / 전체 비중 = AI 도움 비율

**쉽게 말하면:**
```python
AIOE = sum(가중 영향도) / sum(Scalar)
     = AI가 도와줄 수 있는 총 비중 / 전체 능력 가중치
     = AI가 도와줄 수 있는 비율
```

In [ ]:
# TODO: 직업별로 집계하고 AIOE 기본 점수를 계산하세요
#
# 힌트 1: df.groupby(['soc_code', 'title']) 사용
# 힌트 2: .agg()로 집계 - wtd_element_impact는 sum, scalar도 sum, ability는 count
# 힌트 3: .reset_index()로 인덱스 리셋
# 힌트 4: 새 컬럼명: total_weighted_impact, ability_base, n_abilities
#
# 변수명: results_df

# 여기에 코드를 작성하세요



In [ ]:
# TODO: AIOE 기본 점수를 계산하세요
#
# 힌트: ability_base_impact = total_weighted_impact ÷ ability_base
#
# 새 컬럼명: ability_base_impact

# 여기에 코드를 작성하세요



In [579]:
# Software Developers 결과 확인
results_df[results_df['title'] == 'Software Developers'][
    ['title', 'total_weighted_impact', 'ability_base', 'ability_base_impact']
]

,title,total_weighted_impact,ability_base,ability_base_impact
112,Software Developers,5.413907,7.905303,0.684845


---

## 7. Step 5: 표준 점수(Z-score)로 변환하기

### 왜 Z-score로 변환하나요?

**현재 문제:**
- AIOE 기본 점수는 **절대값** (예: 0.685)
- 이 숫자만 봐서는 "높은건가 낮은건가?" 판단하기 어려움

**해결책: Z-score 변환**
- 각 점수를 **상대적 위치**로 변환
- "다른 직업들과 비교해서 어느 정도인가?"를 보여줌

### 3명 학생의 시험 점수를 생각해봅시다

**점수:**
- 철수: 70점
- 영희: 85점  
- 민수: 90점
- **평균**: 81.67점

**절대 점수만 보면:**
- 철수가 얼마나 뒤처졌는지 애매함

**Z-score로 변환하면:**
- 철수: -1.2 (평균보다 낮음)
- 영희: +0.3 (평균보다 약간 높음)
- 민수: +0.9 (평균보다 높음)

→ **상대적 위치가 명확해집니다!**

### Z-score 공식

**Z-score = (내 점수 - 평균) / 표준편차**

**해석:**
- **Z = 0**: 평균 (중간)
- **Z > 0**: 평균보다 높음
- **Z < 0**: 평균보다 낮음
- **Z > 1**: 상위 16% (높음)
- **Z > 2**: 상위 2.5% (매우 높음)

In [ ]:
# TODO: Z-score로 표준화하고 순위를 계산하세요
#
# 힌트 1: stats.zscore()를 ability_base_impact에 적용
# 힌트 2: .rank(ascending=False, method='min')로 순위 계산
# 힌트 3: .astype(int)로 정수형 변환
#
# 새 컬럼명: aioe, rank

# 여기에 코드를 작성하세요



In [581]:
results_df[['title', 'ability_base_impact', 'aioe', 'rank']].tail()

,title,ability_base_impact,aioe,rank
889,Gas Compressor and Gas Pumping Station Operators,0.511348,-0.975348,685
890,"Pump Operators, Except Wellhead Pumpers",0.506942,-1.027508,704
891,Wellhead Pumpers,0.491941,-1.205123,763
892,Refuse and Recyclable Material Collectors,0.467224,-1.497766,846
893,"Tank Car, Truck, and Ship Loaders",0.465940,-1.512960,850


## 8. 결과 저장

In [587]:
# 직업별 AIOE 계산
results_df = df.groupby(['soc_code', 'title']).agg(
    total_weighted_impact=('wtd_element_impact', 'sum'),
    ability_base=('scalar', 'sum'),
    n_abilities=('ability', 'count')
).reset_index()

results_df['ability_base_impact'] = results_df['total_weighted_impact'] / results_df['ability_base']
results_df['aioe'] = stats.zscore(results_df['ability_base_impact'])
results_df['rank'] = results_df['aioe'].rank(ascending=False, method='min').astype(int)

In [ ]:
results_df.head()

,soc_code,title,total_weighted_impact,ability_base,n_abilities,ability_base_impact,aioe,rank
0,11-1011.00,Chief Executives,7.771722,11.088054,52,0.700909,1.268992,106
1,11-1011.03,Chief Sustainability Officers,6.363911,8.992497,52,0.707691,1.349286,69
2,11-1021.00,General and Operations Managers,5.729594,8.417660,52,0.680664,1.029288,199
3,11-2011.00,Advertising and Promotions Managers,6.281507,9.100466,52,0.690240,1.142671,156
4,11-2021.00,Marketing Managers,6.247070,9.065697,52,0.689089,1.129040,161


---

## 9. 최종 요약

### 이번 노트북에서 배운 것

**AIOE 계산 과정 (5단계):**

1. **정규화** - Importance(1-5), Level(0-7)을 0-1 범위로 변환
2. **Scalar 계산** - Level × Importance (능력의 가중치)
3. **가중 영향도 계산** - Scalar × Beta (AI의 실제 영향도)
4. **직업별 집계** - 52개 능력을 합산하여 직업당 1개 점수 산출
5. **Z-score 변환** - 상대적 위치를 알 수 있도록 표준화

### 핵심 공식

$$
AIOE_j = \frac{\sum_{a=1}^{52} (Level_{ja} \times Importance_{ja} \times Beta_a)}{\sum_{a=1}^{52} (Level_{ja} \times Importance_{ja})}
$$

**해석:** 전체 능력 중 AI가 도와줄 수 있는 능력의 비율

### 결과 데이터

**저장 위치:** `../datas/processed/aioe_scores.csv`

**데이터 구조:**
- 894개 직업 × 7개 컬럼
- 주요 컬럼: `SOC_Code`, `Title`, `ability_base_impact`, `aioe`, `rank`

**AIOE 범위:**
- 최고: +1.79 (Z-score)
- 최저: -1.97 (Z-score)
- 평균: 0.00 (Z-score 특성)

### 다음 단계

**03_AIOE_EDA.ipynb**에서:
- AIOE 점수 분포 시각화
- 직업군별 비교 분석
- Software Developers 사례 분석
- 상관관계 탐색